In [1]:
!pip install transformers[torch] -q

In [2]:
!pip install datasets -q --upgrade

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf 23.8.0 requires cubinlinker, which is not installed.
cudf 23.8.0 requires cupy-cuda11x>=12.0.0, which is not installed.
cudf 23.8.0 requires ptxcompiler, which is not installed.
cuml 23.8.0 requires cupy-cuda11x>=12.0.0, which is not installed.
dask-cudf 23.8.0 requires cupy-cuda11x>=12.0.0, which is not installed.
apache-beam 2.46.0 requires dill<0.3.2,>=0.3.1.1, but you have dill 0.3.7 which is incompatible.
apache-beam 2.46.0 requires pyarrow<10.0.0,>=3.0.0, but you have pyarrow 16.1.0 which is incompatible.
beatrix-jupyterlab 2023.128.151533 requires jupyterlab~=3.6.0, but you have jupyterlab 4.0.11 which is incompatible.
cudf 23.8.0 requires cuda-python<12.0a0,>=11.7.1, but you have cuda-python 12.3.0 which is incompatible.
cudf 23.8.0 requires pandas<1.6.0dev0,>=1.3, but you have pandas 2.1.4 which is i

In [3]:
!pip install evaluate -q

In [4]:
# !pip install wandb -q

In [5]:
import os
import gc
import math
import librosa
import numpy as np
import pandas as pd
import librosa as lb
import matplotlib.pyplot as plt

from scipy import signal
from scipy.fft import fftshift

from collections import defaultdict
from itertools import islice
from typing import Any
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.metrics import f1_score, precision_score, recall_score

import torch
import torchaudio

from IPython import display
from torch import nn
from torch.utils.data import DataLoader, Dataset, Subset
from tqdm.auto import trange
from tqdm import tqdm
tqdm.pandas()

import re
import seaborn as sns
from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union
from datasets import (
    load_dataset,
    load_metric,
    load_from_disk,
    Audio,
    concatenate_datasets,
    Dataset
)
from transformers import (
    AutoConfig,
    Wav2Vec2Processor,
    Wav2Vec2Tokenizer,
    Wav2Vec2Config,
    Wav2Vec2FeatureExtractor,
    Wav2Vec2CTCTokenizer,
    Wav2Vec2ForSequenceClassification,
    Wav2Vec2Model,
    TrainingArguments,
    Trainer,
    get_linear_schedule_with_warmup,
    EvalPrediction
)

from transformers.models.wav2vec2.modeling_wav2vec2 import (
    Wav2Vec2PreTrainedModel,
    Wav2Vec2Model
)
import evaluate

2024-05-25 21:51:02.630796: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-25 21:51:02.630916: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-25 21:51:02.767617: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [8]:
import wandb
wandb.login()

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [9]:
import os
os.environ["WANDB_PROJECT"]="thesis_model_batch"
os.environ["WANDB_LOG_MODEL"] = "checkpoint"

In [10]:
import datasets
datasets.__version__

'2.19.1'

In [11]:
train_path = '/kaggle/input/thesis-dataset-full/train'
train_path_aug1 = '/kaggle/input/thesis-dataset-full/train_augs/train'
train_path_aug2 = '/kaggle/input/thesis-dataset-full/train_augs/train 2'
val_path = '/kaggle/input/thesis-dataset-full/data_full/val_full.pickle'
test_path = '/kaggle/input/thesis-dataset-full/data_full/test_full.pickle'
path_to_save = '/kaggle/working/first_model.pth'

model_name = 'facebook/wav2vec2-large-xlsr-53'
run_name = 'basic_model_one_tahn_russian'

In [12]:
train1 = load_from_disk(train_path_aug1)
train2 = load_from_disk(train_path_aug2)

val_pd = pd.read_pickle(val_path)
test_pd = pd.read_pickle(test_path)

Loading dataset from disk:   0%|          | 0/22 [00:00<?, ?it/s]

Loading dataset from disk:   0%|          | 0/22 [00:00<?, ?it/s]

In [13]:
from datasets import concatenate_datasets

train = concatenate_datasets([train1, train2])
labels = train['labels']
labels.count(0), labels.count(1), labels.count(2), labels.count(3)

(19878, 14812, 17495, 13948)

In [17]:
class CustomDataset(Dataset):

    def __init__(self, input_values, labels, audio_paths):
        super().__init__()
        self.input_values = input_values
        self.labels = labels
        self.audio_paths = audio_paths

    def __len__(self):
        return len(self.input_values)

    def __getitem__(self, idx):
        return {'input_values': self.input_values[idx].tolist(), 'labels': self.labels[idx], 'audio_ID': self.audio_paths[idx]}

In [18]:
val = Dataset.from_pandas(val_pd)
test = Dataset.from_pandas(test_pd)

In [20]:
import gc
del val_pd
gc.collect()

80

In [21]:
from torch import Tensor

def collate_fn_pooled_tokens(data):
    input_ids = [torch.tensor(data[i]['input_values']).cuda() for i in range(len(data))]
    attention_mask = [torch.where(input_ids[i]==0, 0, 1).cuda() for i in range(len(input_ids))]
    audio_ids = [data[i]['audio_ID'] for i in range(len(data))]
    if len(data[0]) == 2:
        collated = [input_ids, attention_mask, audio_ids]
        # collated = [input_ids]
    else:
        labels = Tensor([data[i]['labels'] for i in range(len(data))])
        collated = [input_ids, attention_mask, labels, audio_ids]
        # collated = [input_ids, labels]
    return collated

In [23]:
train_dataloader_splitted = DataLoader(train, batch_size=8, collate_fn=collate_fn_pooled_tokens)
val_dataloader_splitted = DataLoader(val, batch_size=8, collate_fn=collate_fn_pooled_tokens)

## Initialize model

In [26]:
from dataclasses import dataclass
from typing import Optional, Tuple
import torch
from transformers.file_utils import ModelOutput


@dataclass
class SpeechClassifierOutput(ModelOutput):
    loss: Optional[torch.FloatTensor] = None
    logits: torch.FloatTensor = None
    hidden_states: Optional[Tuple[torch.FloatTensor]] = None
    attentions: Optional[Tuple[torch.FloatTensor]] = None

In [28]:
# https://github.com/m3hrdadfi/soxan/blob/main/notebooks/Emotion_recognition_in_Greek_speech_using_Wav2Vec2.ipynb

class Wav2Vec2ClassificationHead(nn.Module):
    """Head for wav2vec classification task."""

    def __init__(self, config):
        super().__init__()
        self.dense = nn.Linear(1024, config['hidden_size'])
        self.hidden_dropout = nn.Dropout(config['hidden_dropout'])
        self.final_dropout = nn.Dropout(config['final_dropout'])
        self.out_proj = nn.Linear(config['hidden_size'], config['num_labels'])
        self.sigmoid = nn.Sigmoid()
        self.relu = nn.ReLU()
        self.softmax = nn.Softmax()

    def forward(self, features, **kwargs):
        x = self.hidden_dropout(features)
        x = self.dense(x)
        x = self.relu(x)
        x = self.final_dropout(x)
        x = self.out_proj(x)
        return x


class Wav2VecClassifer(Wav2Vec2PreTrainedModel):

    def __init__(self, w2v_config, config):
        super().__init__(w2v_config)

        self.num_labels = config['num_labels']
        self.pooling_strategy = config['pooling_mode']

        self.wav2vec2 = Wav2Vec2Model.from_pretrained(
            config['model_name'], 
            config=w2v_config)

        self.classifier = Wav2Vec2ClassificationHead(config)#.to(self.device)
        self.loss = nn.CrossEntropyLoss()

        self.post_init()

    def freeze_feature_extractor(self):
        self.wav2vec2.feature_extractor._freeze_parameters()

    def merged_strategy(
            self,
            hidden_states,
            mode="mean"
    ):
        if mode == "mean":
            outputs = torch.mean(hidden_states, dim=1)
        elif mode == "sum":
            outputs = torch.sum(hidden_states, dim=1)
        elif mode == "max":
            outputs = torch.max(hidden_states, dim=1)[0]
        else:
            raise Exception(
                "The pooling method hasn't been defined! Your pooling mode must be one of these ['mean', 'sum', 'max']")

        return outputs

    def forward(
            self,
            input_values,
            labels,
            attention_mask=None,
            device=None,
            output_attentions=False,
            output_hidden_states=False,
            number_of_chunks=None
    ):

        outputs = self.wav2vec2(
            input_values=input_values,
            attention_mask=attention_mask,
            output_attentions=output_attentions,
            output_hidden_states=output_hidden_states
        )
        
        hidden_states = outputs[0]

        if attention_mask is None:
            pooled_output = hidden_states.mean(dim=1)
        else:
            padding_mask = self._get_feature_vector_attention_mask(hidden_states.shape[1], attention_mask)
            padding_mask = padding_mask
            hidden_states[~padding_mask] = -100
            pooled_output = hidden_states.sum(dim=1) / padding_mask.sum(dim=1).view(-1, 1)

        logits = self.classifier(pooled_output)
        preds = logits.view(-1, self.num_labels)
        loss = None
        if labels is not None:
            loss = self.loss(preds, labels.view(-1)) #
        
        output = (preds,) + outputs[2:]
        return ((loss,) + output) if loss is not None else output

In [29]:
config = {
    'output_hidden_states': False,
    'num_labels': 2,
    'hidden_size': 64,
    'final_dropout': 0.1,
    'hidden_dropout': 0.2,
    'pooling_mode': 'mean',
    'model_name': 'jonatasgrosman/wav2vec2-large-xlsr-53-russian',
    'input_size': 1024
}

model_args_w2v2 = {
    "attention_dropout": 0.1,
    "hidden_dropout": 0.1,
    "feat_proj_dropout": 0.0,
    "mask_time_prob": 0.05,
    "layerdrop": 0.1,
    'layer_norm_eps':1e-4,
}

model_args_w2v2_config = Wav2Vec2Config.from_pretrained(
    config['model_name'], 
    attention_dropout=0.1,
    hidden_dropout=0.1,
    feat_proj_dropout=0.0,
    mask_time_prob=0.05,
    layerdrop=0.1,
)

model = Wav2VecClassifer(model_args_w2v2_config, config).cuda()

/opt/conda/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/1.78k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.26G [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
Some weights of Wav2Vec2Model were not initialized from the model checkpoint at jonatasgrosman/wav2vec2-large-xlsr-53-russian and are newly initialized: ['wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original1']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [30]:
model.freeze_feature_extractor()

### Trainer

In [31]:
from dataclasses import dataclass
from typing import Dict, List, Optional, Union
import torch

import transformers
from transformers import Wav2Vec2Processor


@dataclass
class DataCollatorCTCWithPadding:
    """
    Data collator that will dynamically pad the inputs received.
    Args:
        processor (:class:`~transformers.Wav2Vec2Processor`)
            The processor used for proccessing the data.
        padding (:obj:`bool`, :obj:`str` or :class:`~transformers.tokenization_utils_base.PaddingStrategy`, `optional`, defaults to :obj:`True`):
            Select a strategy to pad the returned sequences (according to the model's padding side and padding index)
            among:
            * :obj:`True` or :obj:`'longest'`: Pad to the longest sequence in the batch (or no padding if only a single
              sequence if provided).
            * :obj:`'max_length'`: Pad to a maximum length specified with the argument :obj:`max_length` or to the
              maximum acceptable input length for the model if that argument is not provided.
            * :obj:`False` or :obj:`'do_not_pad'` (default): No padding (i.e., can output a batch with sequences of
              different lengths).
        max_length (:obj:`int`, `optional`):
            Maximum length of the ``input_values`` of the returned list and optionally padding length (see above).
        max_length_labels (:obj:`int`, `optional`):
            Maximum length of the ``labels`` returned list and optionally padding length (see above).
        pad_to_multiple_of (:obj:`int`, `optional`):
            If set will pad the sequence to a multiple of the provided value.
            This is especially useful to enable the use of Tensor Cores on NVIDIA hardware with compute capability >=
            7.5 (Volta).
    """

    processor: Wav2Vec2Processor
    padding: Union[bool, str] = True
    max_length: Optional[int] = None
    max_length_labels: Optional[int] = None
    pad_to_multiple_of: Optional[int] = None
    pad_to_multiple_of_labels: Optional[int] = None

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        input_features = [{"input_values": feature["input_values"]} for feature in features]
        label_features = [feature["labels"] for feature in features]

        d_type = torch.long if isinstance(label_features[0], int) else torch.float

        batch = self.processor.pad(
            input_features,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors="pt",
        )

        batch["labels"] = torch.tensor(label_features, dtype=d_type)
#         print(batch)

        return batch

In [32]:
processor = Wav2Vec2FeatureExtractor.from_pretrained('jonatasgrosman/wav2vec2-large-xlsr-53-russian')
data_collator = DataCollatorCTCWithPadding(processor=processor, padding=True)

preprocessor_config.json:   0%|          | 0.00/262 [00:00<?, ?B/s]

In [33]:
def compute_metrics(p: EvalPrediction):
    preds = p.predictions[0] if isinstance(p.predictions, tuple) else p.predictions
    preds = np.argmax(preds, axis=1)
    labels = p.label_ids
    
    cr = evaluate.load('bstrai/classification_report')
    result = cr.compute(predictions=preds, references=labels, labels=[0, 1, 2, 3])

    res_dict = {}
    for label in result.keys():
        if isinstance(result[label], dict):
            for metric in result[label].keys():
                res_dict[f'{metric}_{label}'] = result[label][metric]
        else:
            res_dict[label] = result[label]

    return res_dict

In [34]:
from transformers import TrainingArguments
import random

def set_seed(seed: int, deterministic: bool = False):
    """
    Helper function for reproducible behavior to set the seed in `random`, `numpy`, `torch` and/or `tf` (if installed).

    Args:
        seed (`int`):
            The seed to set.
        deterministic (`bool`, *optional*, defaults to `False`):
            Whether to use deterministic algorithms where available. Can slow down training.
    """
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    # ^^ safe to call this function even if cuda is not available
    if deterministic:
        torch.use_deterministic_algorithms(True)
        
set_seed(888) #888

In [35]:
num_epochs = 2
warmup_ratio = 0.1
total_steps = len(train_dataloader_splitted) * num_epochs

In [ ]:
# for lr in [1e-3, 2e-3, 1e-4]
lr = 1e-3
config = {
    'output_hidden_states': False,
    'num_labels': 4,
    'hidden_size': 64,
    'final_dropout': 0.1,
    'hidden_dropout': 0.2,
    'pooling_mode': 'mean',
    'model_name': 'jonatasgrosman/wav2vec2-large-xlsr-53-russian',
    'input_size': 1024
}

model_args_w2v2 = {
    "attention_dropout": 0.1,
    "hidden_dropout": 0.1,
    "feat_proj_dropout": 0.0,
    "mask_time_prob": 0.05,
    "layerdrop": 0.1,
    'layer_norm_eps':1e-4,
}

model_args_w2v2_config = Wav2Vec2Config.from_pretrained(
    config['model_name'], 
    attention_dropout=0.1,
    hidden_dropout=0.1,
    feat_proj_dropout=0.0,
    mask_time_prob=0.05,
    layerdrop=0.1,
)

model = Wav2VecClassifer(model_args_w2v2_config, config).cuda()
model.freeze_feature_extractor()

optimizer = torch.optim.AdamW([
        {"params": model.wav2vec2.parameters(), "lr": 3e-4},
    ], lr=lr)

scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=total_steps * warmup_ratio,
    num_training_steps=total_steps
)

training_args = TrainingArguments(
    output_dir=f"/kaggle/working/model_trainer_{lr}",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    evaluation_strategy="steps",
    num_train_epochs=num_epochs,
    fp16=True,
    save_steps=3000,
    eval_steps=1500,
    learning_rate=lr,
    save_total_limit=3,
    report_to='wandb',
    run_name=f"model_2_{lr}_w2v_3e-4_one_value_full_augs_888",
    logging_steps=1000,
    load_best_model_at_end=True,
    seed=888
)

trainer = Trainer(
    model=model,
    data_collator=data_collator,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=train,
    eval_dataset=val,
    tokenizer=processor,
    optimizers=(optimizer, scheduler)
)

trainer.train()

In [ ]:
wandb.finish()